In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(123)
from collections import OrderedDict as od

In [76]:
#number of users for biased dataset
unbiased_count = 70000
#number of users for unbiased dataset
biased_count =70000
#number of rows per group in unbiased dataset
rows_per_category = 800

In [208]:
items_file = '../items.csv'
users_file = '../users.csv' 
interactions_file = '../interactions.csv'
trimmed_data = True
#directory where data will be stored
folder = 'reduced_data/'
test_interaction_file = folder + 'test_interaction.csv'
test_users_file = folder +'test_users.csv'
train_users_file = folder + 'train_users.csv'
#assigned gender file
assigned_users_file = folder + 'assigned_users.csv'
unbiased_test_interactions_file = folder + 'unbiased_test_interactions.csv'
biased_test_interactions_file = folder + 'biased_test_interactions.csv'
train_interactions_file = folder + 'train_interactions.csv'
train_negative_sampling_interactions_file = folder + 'negative_interactions.csv'


In [115]:
items = pd.read_csv(items_file,sep='\t')
users = pd.read_csv(users_file,sep='\t',dtype={'career_level':pd.Int64Dtype(), 'discipline_id':pd.Int64Dtype(), 'industry_id':pd.Int64Dtype(), 'experience_n_entries_class': pd.Int64Dtype(), 'experience_years_experience':pd.Int64Dtype(), 'experience_years_in_current':pd.Int64Dtype(),'edu_degree':pd.Int64Dtype()})
interactions = pd.read_csv(interactions_file,'\t')
users.fillna(100,inplace=True)
items.fillna(-100,inplace=True)

/tmp/ipykernel_348978/784051060.py:3: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  interactions = pd.read_csv(interactions_file,'\t')


In [9]:
career_count = users.groupby('career_level').count().sort_values(by="id").reset_index()
career_count
career_dict = {0 : "unknown",
1 : "Student/Intern",
2 : "Entry Level (Beginner)",
3 : "Professional/Experienced",
4 : "Manager (Manager/Supervisor)",
5 : "Executive (VP, SVP, etc.)",
6 : "Senior Executive (CEO, CFO, President)",
100: "Unknown"}

edu_dict = {0: "unknown",1 : "bachelor",2 : "master",3 : "phd",100:"unknown"}
encoding_dict = {'gender':{1:'male',0:'female'}}

In [10]:
import itertools
from collections import Counter
import math
def attribute_assignment(data,field_dict = {"industry_id" : [0,21], "career_level" : [1,2,3]}, set_attr = 'gender', proportion = [0.8,0.2], choices = [0,1]):
    df = data.copy()
    field_names = field_dict.keys()
    x=[{field : df[field].unique()} for field in field_names]
    combination = list(itertools.product(*list(field_dict.values()))) 
    for c in combination:
      new_ind = None
      for id,field in enumerate(field_names):
            ind = df[field] == c[id]
            if type(new_ind) != type(None):
               new_ind = ind & new_ind
            else:
                  new_ind =ind 
      if sum(new_ind)>0:
        total = sum(new_ind)
        choice_prop= list(zip(choices,proportion))
        final_array=[]
        for item in choice_prop:
          element = item[0]
          prop=item[1]
          arr = [element] * math.ceil(total*prop)
          final_array = final_array+arr
        final_array =np.array(final_array[:total])
        np.random.shuffle(final_array)
        df.loc[new_ind, set_attr] = final_array
    
   
    return df

In [112]:
#users with atleast 3 interactions
users_pop = interactions.user_id.value_counts()
good_users = users_pop[users_pop>2].index

if trimmed_data:
    #users only from 1,8,0,21
    limited_users = users[users['industry_id'].isin([1,8,0,21])]
    limited_users = limited_users[limited_users['id'].isin(good_users)]
else:
    limited_users = users[users['id'].isin(good_users)]



In [124]:
interactions = interactions[interactions.user_id.isin(limited_users.id.values)]

#limited_users = limited_users[limited_users['id'].isin(interactions['user_id'].unique())]   
#interactions = interactions[interactions.user_id.isin(good_users)]
interactions_positive = interactions[interactions['interaction_type'] != 4]
#** reassigning var to users
#users = limited_users[limited_users['id'].isin(interactions['user_id'].unique())]
unbiased_test_users = limited_users.groupby(['career_level','experience_years_experience','industry_id'],as_index=False).apply(lambda x: x.sample(rows_per_category,replace=True,random_state =1))
unbiased_test_users.drop_duplicates(subset=['id'],inplace=True)
unbiased_test_users =unbiased_test_users[:unbiased_count]
train_users = limited_users.drop(unbiased_test_users.index.levels[1])
#train_users.reset_index(inplace=True,drop=True)
unbiased_test_users.reset_index(inplace=True,drop=True)
#train_users.drop_duplicates(subset=['id'],inplace=True)

In [129]:
test_users = train_users.sample(biased_count,random_state=1)
train_users = train_users[~train_users['id'].isin(test_users['id'])]
all_users = pd.concat([test_users,train_users])

In [135]:
split_divide =5
def variable_head(group):
    # Modify this function to get the head size you want
    head_size = max(1,(len(group) //split_divide) )
    return group.head(head_size)
unbiased_test_interaction = interactions_positive[interactions_positive['user_id'].isin(unbiased_test_users['id'])].sort_values(['created_at'],ascending=False).groupby(['user_id'],as_index=False).apply(variable_head)

In [136]:
# split data by taking latest interaction as test data for each user id
biased_test_interaction = interactions_positive[interactions_positive['user_id'].isin(test_users['id'])].sort_values(['created_at'],ascending=False).groupby(['user_id'],as_index=False).apply(variable_head)

In [84]:
# removing biased and unbiased interaction
train_interaction = interactions[~interactions.index.isin(unbiased_test_interaction.droplevel(0).index)]
train_interaction = train_interaction[~train_interaction.index.isin(biased_test_interaction.droplevel(0).index)]
#industry_ids = users['industry_id'].unique()
industry_ids = [1,0,21,8]
career_level = limited_users['career_level'].unique()
experience_years_experience = users['experience_years_experience'].unique()
unbiased_users_test = attribute_assignment(unbiased_test_users,field_dict = od({"industry_id" : industry_ids,"career_level":career_level,"experience_years_experience":experience_years_experience}), set_attr = 'gender', proportion = [0.5,0.5], choices = [0,1])

In [141]:
from collections import OrderedDict as od
#users_1 = attribute_assignment(users,field_dict = od({"industry_id" : [0,21], "career_level" : [4,5,6]}),  set_attr = 'gender', proportion = [0.2,0.8], choices = [0,1])

#proportion of males with lowER years of experience is (much) higher than the proportion of female with low years of experience
users_1 = attribute_assignment(all_users,field_dict = od({"industry_id" : [0,21], "career_level" : [4,5,6],"experience_years_experience" : [0,1,2,3]}), set_attr = 'gender', proportion = [0.2,0.8], choices = [0,1])
#proportion of males with HighER years of experience is lower than the proportion of females with high years of experience
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [0,21], "career_level" : [4,5,6],"experience_years_experience" : [4,5,6,7,100]}), set_attr = 'gender', proportion = [0.8,0.2], choices = [0,1])

#proportion of males with highER years of experience is (much) lower than the proportion of females with high years of experience
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [0,21], "career_level" : [0,1,2,3,100],"experience_years_experience" : [4,5,6,7,100]}), set_attr = 'gender', proportion = [0.8,0.2], choices = [0,1])
#proportion of males with lowER years of experience is higher than the proportion of females with low
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [0,21], "career_level" : [0,1,2,3,100],"experience_years_experience" : [0,1,2,3]}), set_attr = 'gender', proportion = [0.2,0.8], choices = [0,1])

#proportion of females with lowER years of experience is (much) higher than the proportion of males with low years of experience
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [1,8], "career_level" : [4,5,6],"experience_years_experience" : [0,1,2,3]}), set_attr = 'gender', proportion = [0.7,0.3], choices = [0,1])
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [1,8], "career_level" : [4,5,6],"experience_years_experience" : [4,5,6,7,100]}), set_attr = 'gender', proportion = [0.3,0.7], choices = [0,1])


users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [1,8], "career_level" : [0,1,2,3,100],"experience_years_experience" : [0,1,2,3,100]}), set_attr = 'gender', proportion = [0.7,0.3], choices = [0,1])
users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : [1,8], "career_level" : [0,1,2,3,100],"experience_years_experience" : [4,5,6,7]}), set_attr = 'gender', proportion = [0.3,0.7], choices = [0,1])
if ~trimmed_data:
    industry_ids = users['industry_id'].unique()
    users_1 = attribute_assignment(users_1,field_dict = od({"industry_id" : list(set(industry_ids).difference(set([0,21,1,8])))}), set_attr = 'gender', proportion = [0.5,0.5], choices = [0,1])


In [145]:
pd.concat([biased_test_interaction,unbiased_test_interaction]).to_csv(test_interaction_file,header=True,index=False)
train_users.to_csv(train_users_file,header=True,index=False)
test_users.to_csv(test_users_file,header=True,index=False)
pd.concat([users_1,unbiased_users_test]).to_csv(assigned_users_file,header=True,index=False)
unbiased_test_interaction.groupby('user_id')['item_id'].apply(list).reset_index
unbiased_test_interaction.to_csv(unbiased_test_interactions_file,header=True,index=False)
biased_test_interaction.groupby('user_id')['item_id'].apply(list).reset_index
biased_test_interaction.to_csv(biased_test_interactions_file,header=True,index=False)
train_interaction.to_csv(train_interactions_file,header=True)

In [150]:
impressions = pd.read_csv('../impressions.csv','\t')

/tmp/ipykernel_348978/3940554283.py:1: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  impressions = pd.read_csv('../impressions.csv','\t')


In [ ]:
impressions['items'].apply(lambda x: set(x))

In [201]:
x=impressions[impressions['user_id'].isin(limited_users.id.values)]

In [202]:
x['item_id'] = x['items'].apply(lambda x: list(set(x.split(','))))

/tmp/ipykernel_348978/644485269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['item_id'] = x['items'].apply(lambda x: list(set(x.split(','))))


In [203]:
x=x[['user_id','item_id']]

In [205]:
x=x.explode('item_id')

In [206]:
x['interaction_type'] =4

In [209]:
x[:5000000].to_csv(train_negative_sampling_interactions_file,header=True,index=False)